# K-shape algorithm

## Required Python Libraries

In [2]:
import math
import numpy as np
import os
import csv
import time
import multiprocessing
from numpy.random import randint
from numpy.linalg import norm, eigh
from numpy.fft import fft, ifft
from sklearn.base import ClusterMixin, BaseEstimator
from sklearn.metrics import rand_score, normalized_mutual_info_score, adjusted_rand_score

## Functions

### ***Z-score*** Function

Z-score is a mathematical approach to convert a "Generic" Random Variables to a Normal Random Variables. 

For more information about this approach: 
https://it.wikipedia.org/wiki/Standardizzazione_(statistica)

**Function Inputs**
1. ```a```: numpy array containing time-series values;
2. ```axis```: axis used to evaluate mean and standard deviation. For axis we intend rows or columns.
3. ```ddof```: Degrees of freedom correction in the calculation of the standard deviation. Default value equals to 0

**Function returned values**:
1. Normalized Time-Series

**Function Assumptions**:
1. ```axis = 0 ```: normalize always on column axis. This is true beacuse we want to normalize time instants values, to exlude possible outliers or errors

In [3]:
def zscore(a, axis=0, ddof=0):
    a = np.asanyarray(a) #convert the input a in an array
    mns = a.mean(axis=axis) #compute the mean of the time series along the column axis (mean for each time instant)
    sstd = a.std(axis=axis, ddof=ddof) #compute the standard deviation of the array along the column axis (mean for each time instant)

    ## If Condition-->
    ## If axis equals to row (axis = 1) and dimensions of mean array is lower than dimensions of time series array
    ## -----> calculate Z score expanding dimensions of mean and std array on row axis
    ## If axis equals to columns (axis = 0) or dimensions of mean array is equal to dimensions of time series array
    ## -----> calculate Z score as time-series array minus its mean, divided by standard deviation

    if axis and mns.ndim < a.ndim:
        res = ((a - np.expand_dims(mns, axis=axis)) / #compute normalized data and expand the the mean and sd to match the dimension with the array a (the mean and std have 1 dimension less=
               np.expand_dims(sstd, axis=axis))
    else:
        res = (a - mns) / sstd #compute normalized data

    ## nan_to_num method --> replace NaN values with 0 or large integer values for infinite values
    return np.nan_to_num(res)

   


### ***Roll_zeropad*** Function

```roll_zeropad``` is a custom function to shift time series values to the right. The number of positions required by the movement are evaluated with parameter ```shift```.
For each shift, a zero value is included in the time series.

**Function Inputs**
1. ```a```: numpy array containing time-series values;
2. ```shift```: Number of right movements required;
3. ```axis```: axis to consider for right movements.

**Function returned values**:
1. Shifted Time-Series

**Function Assumptions**:
1. ```axis = None ```: time series can be considered as a 1D array and not as a matrix

In [4]:
def roll_zeropad(a, shift, axis=None):
    # asanyarray method --> Convert the input to an ndarray, but pass ndarray subclasses through.
    a = np.asanyarray(a) # Ensure that the input is ndarrary data type

    # If no shift is required (shift = 0), it returns the original values of time series
    if shift == 0:
        return a

    # If axis parameter is not evaluated (axis = None)
    # ------> consider all elements of time series and not a specific dimensions, but this requires a reshape of data at the end
    # If axis is set
    # ------> extracted dimensions of a specific axis (rows or columns) and work on that
    if axis is None:
        n = a.size
        reshape = True
    else:
        n = a.shape[axis]
        reshape = False

    # zeros_like() method --> Return an array of zeros with the same shape and type as a given array.

    # If shift parameter is greater than size of the time series (but this requires that no axis are specified)
    # ------> it evaluates an array of 0 values with the same size of a
    if np.abs(shift) > n:
        res = np.zeros_like(a)
    # If shift parameter is lower than 0 (moving to the left)
    # ------> subtract the shift value to n size of the array
    # ------> extract the indices from 0 to (n-shift) values, considering a plain array (axis = None) or a specific dimensions
    # ------> generate a 0-valued array with size [0, (n-shift)]
    # ------> extract values from (n-shift) position to n position
    # ------> create an array with values in range [(n-shift),n] at the beginning and all 0 values after
    elif shift < 0:
        shift += n
        zeros = np.zeros_like(a.take(np.arange(n-shift), axis))
        res = np.concatenate((a.take(np.arange(n-shift, n), axis), zeros), axis)
    # If shift parameter is in the range [0, size of time series]
    # ------> extract values from (n-shift) position to n position
    # ------> generate a 0-valued array with size [(n-shift), n]
    # ------> extract the indices from 0 to (n-shift) values, considering a plain array (axis = None) or a specific dimensions
    # ------> create an array with 0 value at the beginning and then values in range [0, (n-shift)]
    else:
        zeros = np.zeros_like(a.take(np.arange(n-shift, n), axis))
        res = np.concatenate((zeros, a.take(np.arange(n-shift), axis)), axis)

    # If reshape is required
    # -----> change shape (row, column) of shifted array, considering shape of time series values
    if reshape:
        return res.reshape(a.shape)
    else:
        return res

### ***_ncc_c_3dim*** Function

```_ncc_c_3dim``` is a custom function to evaluate the **array** of NCCc values. \
Each element of the array is associated with a specific shift computed on a specific time series.\
NCCc is a normalization of Cross Correlation value, which represents with a value the similarity between 2 time series\
K-Shape Algorithm uses NCCc to evaluate SBD distance measure which is used to assign the time series to the closest cluster centroid.

**Function Inputs**
1. ```data```: numpy array containing 2 time series to compare;
    * Generally, in the algorithm, X is the cluster centroid while Y is the time series to assign.

**Function returned values**:
1. The optimal value of NCCc between X and Y.

**Function Assumptions**:
1. ```None ```

In [64]:
def _ncc_c_3dim(data):
    # Extract X and Y from data
    # Positions in data array are very important
    x, y = data[0], data[1]

    # np.norm function returns one of the eight matrix norms (X parameter considered is a 2D matrix.)
    # Evaluating axis parameter specifies in which dimension evaluate matrix norm.
    den = norm(x, axis=(0,1)) * norm(y, axis=(0,1))
    #print(den)

    # If the computed denominator of NCCc is so small, it is set to infinite value, in order to ignore NCCc value. 
    if den < 1e-9:
        den = np.inf

    # the variable represents the number of time instants considered for each time series.
    x_len = x.shape[0]
    #print(x_len)

    # This is operation is necessary to improve computational effort of FFT.
    # As the article said, in order to improve FFT performances, Cross Correlation must be an exact power-of-two.
    # The following line of code approximate the result to the next power-of-two value. 
    fft_size = 1 << (2*x_len-1).bit_length()

    # As the paper said, CC can be evaluated as convolution of two time series where one of two sequences is reversed in time domain. 
    # The convolution is computed as Inverse Discrete Fourier Transformer (IDFT) of the product of the individual Discrete Fourier Transforms (DFT) of the time series
    # To reduce computational effort, Fast Fourier Transformer (FFT) substitutes DFT.
    # np.conj function return the complex coniugate of a specified number. 
    # The complex conjugate of a complex number is obtained by changing the sign of its imaginary part.
    cc = ifft(fft(x, fft_size, axis=0) * np.conj(fft(y, fft_size, axis=0)), axis=0)

    # CC is the join, along axis 0 (rows), of two selected sequences
    # The selected sequences are extracted from convolution of two time series (IFFT)
    # CC is an array of complex numbers
    cc = np.concatenate((cc[-(x_len-1):], cc[:x_len]), axis=0)
    #print(cc)
    
    # Return array of NCCc values
    return np.real(cc).sum(axis=-1) / den

### ***_sbd*** Function

```_sbd``` function is used to evaluate Shape Based Distance between two time series\
In particular, SBD is similarity measures applied to two time series.\
SBD values are in range [0,2] with 0 assigning perfect match\
```_sbd``` function does not return similarity value, but it returns the optimal shift y.\
The optimal shift of Y represents the closest shift of Y compared to X.

**Function Inputs**
1. ```x```: reference time series
    * Generally, in the algorithm, X is the cluster centroid.
2. ```y```: compared time series
    * Generally, in the algorithm, X is the cluster centroid.

**Function returned values**:
1. The optimal shift of Y.

**Function Assumptions**:
1. ```None ```

In [67]:
# It is possible that the algorithm can ignore this function
# In fact, if cluster centroids are evaluated as all zeros, the function is not used
def _sbd(x, y):
    # Evaluate NCCc array
    ncc = _ncc_c_3dim([x, y])
    #print(ncc)

    # Find the index where NCCc is maximized (w in the paper)
    idx = np.argmax(ncc)
    #print(ncc[idx])

    # The position w is used to evaluate the optimal shift as w-m
    # The shift index is passed to roll_zeropad function to generate optimal y shift
    yshift = roll_zeropad(y, (idx + 1) - max(len(x), len(y)))
    
    return yshift

### ***collect_shift*** Function

**Function Inputs**
1. ```x```: reference time series
    * Generally, in the algorithm, X is the cluster centroid.
2. ```y```: compared time series
    * Generally, in the algorithm, X is the cluster centroid.

**Function returned values**:
1. The optimal shift of Y.

**Function Assumptions**:
1. ```None ```

In [7]:
def collect_shift(data):
    x, cur_center = data[0], data[1]
    if np.all(cur_center==0):
        return x
    else:
        return _sbd(cur_center, x)

In [8]:
def _extract_shape(idx, x, j, cur_center):
    _a=[]
    for i in range(len(idx)):
        if idx[i] == j:
            _a.append(collect_shift([x[i], cur_center]))

    a = np.array(_a)
    
    if len(a) == 0:
        indices = np.random.choice(x.shape[0], 1)
        return np.squeeze(x[indices].copy())
        #return np.zeros((x.shape[1]))

    columns = a.shape[1]
    y = zscore(a, axis=1, ddof=1)

    s = np.dot(y[:, :, 0].transpose(), y[:, :, 0])
    p = np.empty((columns, columns))
    p.fill(1.0/columns)
    p = np.eye(columns) - p
    m = np.dot(np.dot(p, s), p)

    _, vec = eigh(m)
    centroid = vec[:, -1]

    finddistance1 = np.sum(np.linalg.norm(a - centroid.reshape((x.shape[1], 1)), axis=(1, 2)))
    finddistance2 = np.sum(np.linalg.norm(a + centroid.reshape((x.shape[1], 1)), axis=(1, 2)))

    if finddistance1 >= finddistance2:
        centroid *= -1

    return zscore(centroid, ddof=1)

In [9]:
def _kshape(x, k, centroid_init='zero', max_iter=100, n_jobs=1):
    m = x.shape[0]
    idx = randint(0, k, size=m)
    if centroid_init == 'zero':
        centroids = np.zeros((k, x.shape[1], x.shape[2]))
    elif centroid_init == 'random':
        indices = np.random.choice(x.shape[0], k)
        centroids = x[indices].copy()
    distances = np.empty((m, k))
    
    for it in range(max_iter):
        old_idx = idx

        for j in range(k):
            for d in range(x.shape[2]):
                centroids[j, :, d] = _extract_shape(idx, np.expand_dims(x[:, :, d], axis=2), j, np.expand_dims(centroids[j, :, d], axis=1))
                #centroids[j] = np.expand_dims(_extract_shape(idx, x, j, centroids[j]), axis=1)

        pool = multiprocessing.Pool(n_jobs)
        args = []
        for p in range(m):
            for q in range(k):
                args.append([x[p, :], centroids[q, :]])
        result = pool.map(_ncc_c_3dim, args)
        pool.close()
        r = 0
        for p in range(m):
            for q in range(k):
                distances[p, q] = 1 - result[r].max()
                r = r + 1

        idx = distances.argmin(1)
        if np.array_equal(old_idx, idx):
            break

    return idx, centroids

In [10]:
def kshape(x, k, centroid_init='zero', max_iter=100):
    idx, centroids = _kshape(np.array(x), k, centroid_init=centroid_init, max_iter=max_iter)
    clusters = []
    for i, centroid in enumerate(centroids):
        series = []
        for j, val in enumerate(idx):
            if i == val:
                series.append(j)
        clusters.append((centroid, series))

    return clusters

In [11]:
class KShapeClusteringCPU(ClusterMixin,BaseEstimator):
    labels_= None
    centroids_ = None

    def __init__(self,n_clusters, centroid_init='zero', max_iter=100, n_jobs=None):
        self.n_clusters = n_clusters
        self.centroid_init = centroid_init
        self.max_iter = max_iter
        if n_jobs is None:
            self.n_jobs=1
        elif n_jobs == -1:
            self.n_jobs = multiprocessing.cpu_count()
        else:
            self.n_jobs=n_jobs
        


    def fit(self,X,y=None):
        clusters = self._fit(X,self.n_clusters, self.centroid_init, self.max_iter,self.n_jobs)
        self.labels_ = np.zeros(X.shape[0])
        self.centroids_ =np.zeros((self.n_clusters, X.shape[1], X.shape[2]))
        for i in range(self.n_clusters):
            self.labels_[clusters[i][1]] = i
            self.centroids_[i]=clusters[i][0]
        return self

    def predict(self, X):
        labels, _ = self._predict(X,self.centroids_)
        return labels
        
    
    def _predict(self,x, centroids):
        m = x.shape[0]
        idx = randint(0, self.n_clusters, size=m)
        distances = np.empty((m, self.n_clusters))
        

    
        pool = multiprocessing.Pool(self.n_jobs)
        args = []
        for p in range(m):
            for q in range(self.n_clusters):
                args.append([x[p, :], centroids[q, :]])
        result = pool.map(_ncc_c_3dim, args)
        pool.close()
        r = 0
        for p in range(m):
            for q in range(self.n_clusters):
                distances[p, q] = 1 - result[r].max()
                r = r + 1
    
        idx = distances.argmin(1)

        return idx, centroids
    
    
    def _fit(self,x, k, centroid_init='zero', max_iter=100,n_jobs=1):
        idx, centroids = _kshape(np.array(x), k, centroid_init=centroid_init, max_iter=max_iter, n_jobs=n_jobs)
        clusters = []
        for i, centroid in enumerate(centroids):
            series = []
            for j, val in enumerate(idx):
                if i == val:
                    series.append(j)
            clusters.append((centroid, series))
    
        return clusters

## Correlated Functions

```ClusterDataLoader``` class is used to read dataset input files and convert them in a set of arrays. \
The function will load every files (both TRAIN and TEST files) in a specific path. \
Every files must be a CSV files with comma separator\
\
Each row of the file represents a time-series and it contains:
* The first element is cluster which time-series joins;
* All successive elements are measurements of each time-series. Each measurements is associated with a specific time instants. 

It is important to highlight that there aren't any information about time-series or time instants order. \
We suppose that the order of time-series is not important for K-shape Clustering Algorithm.

In [12]:
class ClusterDataLoader:
    def __init__(self, dataset_path):
        self.path = dataset_path

    def load(self, sub_dataset_name):
        ts, labels = [], []
        for mode in ['_TRAIN']:
            with open(os.path.join(self.path, sub_dataset_name, sub_dataset_name + mode)) as csv_file:
                lines = csv.reader(csv_file, delimiter=',')
                for line in lines:
                    ts.append([float(x) for x in line[1:]])
                    labels.append(int(line[0])-1)

        if min(labels) == 1:
            labels = labels - 1
        if min(labels) == -1:
            labels = labels + 1

        return np.array(ts), np.array(labels), int(len(set(labels)))

## K-Shape Execution

Load a subset (100 hundred rows) of ```CROP``` dataset

In [13]:
DATASET_PATH = 'dataset/univariate_example/'
DATASET_NAME = 'Crop'

dataloder = ClusterDataLoader(DATASET_PATH)

In [14]:
ts, labels, num_clusters = dataloder.load(DATASET_NAME)

### Dataset Information

In [15]:
print("-------------------------\nTimeseries Dataset\n-------------------------\n")
print(ts)

-------------------------
Timeseries Dataset
-------------------------

[[-1.04911634 -0.96685745 -0.88459857 ... -0.51685297 -0.59911186
  -0.68137074]
 [-1.10145498 -1.1413895  -1.18132403 ... -1.02158593 -1.10544843
  -1.18931093]
 [-0.80873226 -0.98601697 -1.16330168 ... -0.78711217 -0.70063182
  -0.61415147]
 ...
 [-1.62102196 -1.41436412 -1.20770628 ... -1.28672251 -0.75184341
  -0.2169643 ]
 [-1.0246006  -1.48431623 -1.36938732 ... -1.04614977 -0.39967465
   0.24680046]
 [-1.10748362 -1.14969269 -1.19190177 ... -0.67695102 -0.19154663
   0.29385776]]


In [16]:
print("-------------------------\nTimeseries Dataset Dimensions\n-------------------------\n")
print(ts.shape)

-------------------------
Timeseries Dataset Dimensions
-------------------------

(100, 46)


In [17]:
print("-------------------------\nClusters Dataset\n-------------------------\n")
print(labels)

-------------------------
Clusters Dataset
-------------------------

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [18]:
print("-------------------------\nClusters Dataset Dimensions\n-------------------------\n")
print(labels.shape)

-------------------------
Clusters Dataset Dimensions
-------------------------

(100,)


In [19]:
print(num_clusters)

1


### K-Shape Single Execution

In [65]:
start_time = time.time()

ksc = KShapeClusteringCPU(n_clusters=num_clusters,max_iter=2,n_jobs=-1)
ksc.fit(np.expand_dims(ts, axis=2))
print(f' Execution time: {time.time() - start_time}')

 Execution time: 0.1790626049041748


## Specific Tests

### Monovariate Time Series

Monovariate time-series can be represented as a matrix NxM with:
- N (rows) as the number of time-series. Each row represent a time series which can be connected to a specific date 
- M (columns) as time instants considered for each time series. Each time instants respects a frequency of analysis 

In [21]:
#Example of monovariate array of time series: 4 time series and 3 time instants
##
a_monovariate = np.array([[10, 12, 31],
              [4, 9, 10],
              [70, 81, 2],
              [12, 112, 12]])

### Multivariate Time Series

Multivariate time-series can be represented as a matrix NxM with:
- N (rows) as the number of time-series. Each row represent a time series which can be connected to a specific date 
- M (columns) as time instants considered for each time series. Each time instants respects a frequency of analysis 

The main difference with monovariate time-series is that each cell of the matrix is an array of values. The number of elements of the array can be considered as the number of variables involved, in addition to timestamp

In [22]:
#Example of multivariate array of time series:

#2 time series, with 3 time instants and 2 variables

a_multivariate = np.array([[[11, 1000], [25, 30], [350, 40]],
                           [[45, 0.2], [1, 0.111], [65, 70]],
                           [[17, 59], [11, 14], [45, 87]]])

#compute the mean for each time instant
print("Mean\n-----------")
print(a_multivariate.mean(axis=0))
print("Std\n-----------")
print(a_multivariate.std(axis=0))

Mean
-----------
[[ 24.33333333 353.06666667]
 [ 12.33333333  14.70366667]
 [153.33333333  65.66666667]]
Std
-----------
[[ 14.81740718 458.080352  ]
 [  9.84321537  12.21227362]
 [139.30382463  19.43078886]]


The output can be explained as:
- Cell 0,0 ==> mean (or std) of first elements of first time instant;
- Cell 0,1 ==> mean (or std) of second elements of first time instant;
- Cell 1,0 ==> mean (or std) of first elements of second time instant;
....

So the output is a matrix Nx(M-1)

### Multivariate Time Series - zScore

In [23]:
#we apply the zscore function
zscore_result = zscore(a_multivariate)
print(zscore_result)

[[[-0.89984254  1.41227042]
  [ 1.28684238  1.25253772]
  [ 1.41178225 -1.32092767]]

 [[ 1.39475594 -0.77031609]
  [-1.15138528 -1.19491809]
  [-0.63410559  0.22301376]]

 [[-0.4949134  -0.64195433]
  [-0.13545709 -0.05761963]
  [-0.77767666  1.09791391]]]


### Monovariate Time Series - zScore

In [24]:
#we apply the zscore function
zscore_result = zscore(a_monovariate, axis=0)
print(zscore_result)


[[-0.52393683 -0.93494794  1.62139887]
 [-0.74848119 -1.00253454 -0.35247801]
 [ 1.72150673  0.61954382 -1.10443111]
 [-0.44908871  1.31793866 -0.16448974]]


### Monovariate Time Series - Shifting

In [25]:
print(a_monovariate)

[[ 10  12  31]
 [  4   9  10]
 [ 70  81   2]
 [ 12 112  12]]


In [26]:
a_monovariate_shifted = roll_zeropad(a_monovariate, 1)
print(a_monovariate_shifted)

[[  0  10  12]
 [ 31   4   9]
 [ 10  70  81]
 [  2  12 112]]


Evaluating parameter ```shift``` to 1, the function shifts time series to the right of one position

In [27]:
a_monovariate_shifted = roll_zeropad(a_monovariate, -1)
print(a_monovariate_shifted)

[[ 12  31   4]
 [  9  10  70]
 [ 81   2  12]
 [112  12   0]]


Evaluating parameter ```shift``` to -1, the function shifts time series to the left of one position

In [28]:
a_monovariate_shifted = roll_zeropad(a_monovariate, 5)
print(a_monovariate_shifted)

[[ 0  0  0]
 [ 0  0 10]
 [12 31  4]
 [ 9 10 70]]


In [29]:
a_monovariate_shifted = roll_zeropad(a_monovariate, -3)
print(a_monovariate_shifted)

[[  4   9  10]
 [ 70  81   2]
 [ 12 112  12]
 [  0   0   0]]


### Multivariate Time Series - Shifting

In [30]:
print(a_multivariate)

[[[1.10e+01 1.00e+03]
  [2.50e+01 3.00e+01]
  [3.50e+02 4.00e+01]]

 [[4.50e+01 2.00e-01]
  [1.00e+00 1.11e-01]
  [6.50e+01 7.00e+01]]

 [[1.70e+01 5.90e+01]
  [1.10e+01 1.40e+01]
  [4.50e+01 8.70e+01]]]


In [31]:
a_multivariate_shifted = roll_zeropad(a_multivariate, 1)
print(a_multivariate_shifted)

[[[0.00e+00 1.10e+01]
  [1.00e+03 2.50e+01]
  [3.00e+01 3.50e+02]]

 [[4.00e+01 4.50e+01]
  [2.00e-01 1.00e+00]
  [1.11e-01 6.50e+01]]

 [[7.00e+01 1.70e+01]
  [5.90e+01 1.10e+01]
  [1.40e+01 4.50e+01]]]


The shifting works across the same row. Considering that each element of row has an array of values, which indexes means different variables to analyze (temp, humidity, etc), this can generate misunderstanding data because the operation changes the variable associated to data value

### Monovariate Time Series - NCCc

In [32]:
x = np.array([[2,5]])
y = np.array([[2,5]])

nccc = _ncc_c_3dim([x,y])

print(nccc)

[1. 0. 1.]


In [33]:
sbd = _sbd(x,y)

print(f"SBD: {sbd}")

SBD: [[2 5]]


### FFT

In [34]:
x_len = 300
xlen_trans = 2*x_len-1
print(xlen_trans)

599


In [35]:
print(xlen_trans.bit_length())

10


In [36]:
print (1 << xlen_trans.bit_length())

1024


In [37]:
print(2 ** 11)

2048


In [38]:
print(fft([2,5],))

[ 7.+0.j -3.+0.j]


In [39]:
print(np.zeros((2, 2, 3)))

[[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]]
